In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

Objective function:

$$
\begin{align}
F(\beta)=\frac{1}{n}\sum_{i=1}^n \log(1+\exp(-y_ix_i^\top \beta)) + \lambda||\beta||_2^2
\end{align}
$$

Find 

$$
 F(\beta^*)=\min_{\beta \in \mathbb{R}^d}F(\beta)
$$

In [3]:
def obj_function(X, y, beta, lamb):
    
    n = len(X)
    f1 = np.sum(np.log(1+np.exp(-y*np.dot(beta, X.T)))) * (1/n)
    f2 = np.linalg.norm(beta) ** 2*lamb
    
    return f1 + f2

1. Assume that $d=1$ and $n=1$. The sample is then of size 1 and boils down to just $(x, y)$. The function $F$ writes simply as

$$
F(\beta) = log(1+\exp(-yx\beta)) + \lambda\beta^2
$$

Compute and write down the gradient $\nabla F$ of $F$.

$$\begin{align}
F(\beta) &= log(1+\exp(-yx\beta)) + \lambda\beta^2 \\
\frac{d}{d\beta}F(\beta)&=\frac{1}{1+ \exp(-yx\beta)}\times \exp(-yx\beta) \times (-yx) + 2\lambda\beta\\
&=-yx\frac{\exp(-yx\beta)}{1+\exp(-yx\beta)} + 2\lambda\beta\\
\end{align}
$$

In [4]:
def compgrad_1(X, y, beta, lamb):
    
    num = np.exp(-y*X*beta)
    denom = 1+np.exp(-y*X*beta)
    
    #return -y*X*(np.exp(-y*x*beta)/1+np.exp(-y*x*beta))+2*lamb*beta
    
    return (-y*X*num/denom) + (2*lamb*beta)

2. Assume now that $d>1$ and $n>1$. Using the previous result and the linearity of differentiation, compute and write down the gradient $\nabla F(\beta)$ of $F$.

$$\begin{align}
F(\beta)&=\frac{1}{n}\sum_i^n\log(1+\exp(-y_ix_i^\top \beta)) + \lambda||\beta||_2^2 \\
\nabla F(\beta)&=\frac{1}{n}\sum_i^n\nabla\underbrace{ \log(1+\exp(-y_ix_i^\top \beta))}_1 + \underbrace{\nabla \lambda||\beta||_2^2}_2 \\
\end{align}
$$
1.
$$
\begin{align}
\scriptsize{\frac{d}{du}\log(u(x))=\frac{1}{\log(u)}\times u'(x)} \\
\nabla\log(1+\exp(-y_ix_i^\top \beta))&=\frac{1}{1+\exp(-y_ix_i^\top \beta)}\times \exp(-y_ix_i^\top \beta) \times -y_ix_i \\
&=-y_ix_i\times \frac{\exp(-y_ix_i^\top \beta)}{1+\exp(-y_ix_i^\top \beta)}
\end{align}$$
2.
$$
\begin{align}
\nabla \lambda||\beta||_2^2&=\lambda \beta^\top \beta\\
&=2\lambda\beta
\end{align}
$$
Bringing both components together
$$
\begin{align}
\nabla F(\beta)=\frac{1}{n}\sum_i^n-y_ix_i\frac{\exp(-y_ix_i^\top \beta)}{1+\exp(-y_ix_i^\top \beta)}+2\lambda\beta
\end{align}
$$

3. Consider the `smarket` dataset from *Introduction to Statistical Learning*. Download the data: 
    
    This dataset contains trading information for the S&P 500 over 1250 days from 2001 to 2005. For each date we have the percent return from the previous 5 days (the `Lag` features), the volume of shares traded (in billions), the percent return on the date itself (`Today`), and whether the market moved up or down (`Direction`). We will apply our gradient descent and fast gradient descent algorithms to fit a logistic regression model for the binary outcome `Direction` based on the features `Lag1`, `Lag2`, and `Volume`.

In [5]:
file = 'https://raw.githubusercontent.com/JWarmenhoven/ISLR-python/master/Notebooks/Data/Smarket.csv'
smarket = pd.read_csv(file, sep=',', header=0, index_col=0)
smarket.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
1,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
3,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
4,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
5,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


4. Construct the matrix of the features and response. Transform the response to a vector with entries in $\{+1,-1\}$, corresponding to whether `Direction` is 'Up' or 'Down', respectively. Split the data into train and test sets (80/20 split) and standardize the features.

In [6]:
smarket['response'] = [1 if x == 'Up' else -1 for x in smarket['Direction']]

X = np.array(smarket.iloc[:,np.r_[1:3,6]])
y = np.array(smarket.response)

print('X.shape: ', X.shape)
print('y.shape: ', y.shape)

X.shape:  (1250, 3)
y.shape:  (1250,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

print('X_train.shape: ', X_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (1000, 3)
X_test.shape:  (250, 3)
y_train.shape:  (1000,)
y_test.shape:  (250,)


In [8]:
Xscale = preprocessing.StandardScaler().fit(X_train)
Xs = Xscale.transform(X_train)

print('Xs.shape: ', Xs.shape)

Xs.shape:  (1000, 3)


5. Write a function *computegrad* that computes and returns $\nabla F(\beta)$ for any $\beta$.

In [10]:
m = LogisticRegression().fit(Xs, y_train)
beta_test = m.coef_

In [219]:
def computegrad(X, y, beta, lamb):
    """
    Compute the gradient of the objective function for n>1 and d>1
    """
    n = len(X)
    frac = (np.exp(-y*np.dot(beta, X.T)))/(1+np.exp(-y*np.dot(beta, X.T)))
    f1 = np.sum(frac.dot(-y_train)*Xs)
    #f1 = np.sum(np.dot(-y, np.dot(X.T, frac)))/n
    f2 = 2*lamb*beta
    
    return f1 + f2

In [216]:
frac = (np.exp(-y_train*np.dot(beta_test, Xs.T)))/(1+np.exp(-y_train*np.dot(beta_test, Xs.T)))
np.sum(frac.dot(-y_train)*Xs)+(2*lamb*beta_test)

array([[-0.10264927, -0.05967046,  0.01436939]])

In [214]:
np.sum(frac.dot(-y_train)*Xs)

-5.5138116294983774e-11

In [215]:
beta_test

array([[-0.10264927, -0.05967046,  0.01436939]])

In [208]:
e_ = (np.multiply(-y_train, np.dot(Xs, beta_init)))

In [220]:
def f(beta):
    return obj_function(Xs, y_train, beta, lamb)

def f_grad(beta):
    return computegrad(Xs, y_train, beta, lamb)

f_grad(beta_test)

array([[-0.10264927, -0.05967046,  0.01436939]])

6. Write a function *backtracking* that implements the backtracking rule.

In [221]:
def backtracking(eta_init, decay_rate, prop_constant, f, f_grad, beta, p):
    
    eta = eta_init
    
    def sufficient_decrease(eta):
        lhs = f(beta + eta * p) - f(beta)
        rhs = prop_constant * eta * np.dot(f_grad(beta), p)
        #print('lhs: ', lhs, ' rhs: ', rhs)
        return lhs <= rhs
    
    while not sufficient_decrease(eta):
        eta *= decay_rate
        
    return eta

7. Write a function *graddescent* that implements the gradient descent algorithm with the backpacking rule to tune the step-size. The function *graddescent* calls the *computegrad* and *backtracking* as subroutines. The function takes as input the initial point, the initial step-size value, and the target accuracy $\epsilon$. The stopping criterion is $||\nabla F||\leq \epsilon$.

In [222]:
def graddescent(beta_init, eta_init, decay_rate, prop_constant, target_accuracy, lamb):
    
    beta_t = beta_init
    betas = [beta_t]
    eta = eta_init
    
    def f(beta):
        
        return obj_function(Xs, y_train, beta, lamb)
    
    def f_grad(beta):
        
        return computegrad(Xs, y_train, beta, lamb)
    
    f_norm = np.linalg.norm(f_grad(beta_t))
    
    while not f_norm <= target_accuracy:
        
        #p = -f_grad(beta_t)
        eta = backtracking(eta_init, decay_rate, prop_constant, f, f_grad, beta_t, -f_grad(beta_t))
        beta_t = beta_t + eta * -f_grad(beta_t)
        #print('eta: ', eta)
        betas.append(beta_t)
        #print(beta_t)
        f_norm = np.linalg.norm(f_grad(beta_t))
        #print('fnorm: ',f_norm)
    
    return betas    

In [223]:
eta_init = 1
decay_rate = 0.8
beta_init = np.ones(3)
prop_constant = 0.3
#1/(np.max(np.linalg.eigh((1/len(Xs))*np.dot(Xs.T,Xs))[0])+lamb)
lamb = 0.5
target_accuracy = 0.05

graddescent(beta_init, eta_init, decay_rate, prop_constant, target_accuracy, lamb)

[array([1., 1., 1.]), array([5.51381163e-11, 5.51381163e-11, 5.51381163e-11])]

In [224]:
L = 1/(np.max(np.linalg.eigh((1/len(Xs))*np.dot(Xs.T,Xs))[0])+lamb)

In [225]:
graddescent(beta_init, eta_init, decay_rate, prop_constant, target_accuracy, lamb)

[array([1., 1., 1.]), array([5.51381163e-11, 5.51381163e-11, 5.51381163e-11])]

8. Write a function *fastgradalgo* that implements the fast gradient algorithm described in Algorithm ???. The function *fastgradalgo* calls *computegrad* and *backtracking* as subroutines. The function takes as input the initial step-size value for the backtracking rule and the target accuracy $\epsilon$. The stopping criterion is $||\nabla F||\leq\epsilon$.

In [226]:
def fastgradalgo(beta_init, eta_init, decay_rate, prop_constant, target_accuracy, lamb):
    
    eta = eta_init
    beta_t = beta_init
    betas = [beta_t]
    theta_t = np.zeros(beta_t.shape[0])
    t = 0
    
    def f(beta):
        
        return obj_function(Xs, y_train, beta, lamb)
    
    def f_grad(beta):
        
        return computegrad(Xs, y_train, beta, lamb)
    
    f_norm = np.linalg.norm(f_grad(beta_t))
    
    while not f_norm <= target_accuracy:
        
        beta_1 = theta_t - eta * f_grad(theta_t)
        theta_t = beta_1 + np.dot((t/(t+3)), (beta_1 - beta_t))
        betas.append(beta_1)
        eta = backtracking(eta, decay_rate, prop_constant, f, f_grad, beta_t, -f_grad(beta_t))
        beta_t = beta_1
        t += 1
        f_norm = np.linalg.norm(f_grad(beta_t))
        #print('eta: ', eta)
        #print('theta_t', theta_t)
        #print('fnorm: ',f_norm)
        #print('f_norm: ', f_norm)
        
    return betas

In [227]:
fastgradalgo(beta_init, eta_init, decay_rate, prop_constant, target_accuracy, lamb)

[array([1., 1., 1.]), array([4.17799129e-11, 4.17799129e-11, 4.17799129e-11])]

9. Initialize the step size to $\eta_0=0.1$. Set the target accuracy to $\epsilon=1\times10^{-5}$ Run *graddescent* and *fastgradalgo* on the training set of the smarket dataset for $\lambda=0.5$. Plot the curve of the objective values $F(\beta_t)$ for both algorithms versus the iteration counter $t$ (use different colors). What do you observe? Note: Remember that the scikit-learn penalizes the logistic regression objective differently from our formulation above. You will need to find the setting of their $C$ parameter that corresponds to a given choice of $\lambda$ in our definition.

In [234]:
eta_init = 0.1
decay_rate = 0.7
beta_init = np.ones(3)
prop_constant = 0.4
#1/(np.max(np.linalg.eigh((1/len(Xs))*np.dot(Xs.T,Xs))[0])+lamb)
lamb = 0.5
target_accuracy = 10**-5

In [235]:
#fastgradalgo(beta_init, eta_init, decay_rate, prop_constant, target_accuracy, lamb)
graddescent(beta_init, eta_init, decay_rate, prop_constant, target_accuracy, lamb)

[array([1., 1., 1.]),
 array([0.9, 0.9, 0.9]),
 array([0.81, 0.81, 0.81]),
 array([0.729, 0.729, 0.729]),
 array([0.6561, 0.6561, 0.6561]),
 array([0.59049, 0.59049, 0.59049]),
 array([0.531441, 0.531441, 0.531441]),
 array([0.4782969, 0.4782969, 0.4782969]),
 array([0.43046721, 0.43046721, 0.43046721]),
 array([0.38742049, 0.38742049, 0.38742049]),
 array([0.34867844, 0.34867844, 0.34867844]),
 array([0.3138106, 0.3138106, 0.3138106]),
 array([0.28242954, 0.28242954, 0.28242954]),
 array([0.25418658, 0.25418658, 0.25418658]),
 array([0.22876792, 0.22876792, 0.22876792]),
 array([0.20589113, 0.20589113, 0.20589113]),
 array([0.18530202, 0.18530202, 0.18530202]),
 array([0.16677182, 0.16677182, 0.16677182]),
 array([0.15009464, 0.15009464, 0.15009464]),
 array([0.13508517, 0.13508517, 0.13508517]),
 array([0.12157665, 0.12157665, 0.12157665]),
 array([0.10941899, 0.10941899, 0.10941899]),
 array([0.09847709, 0.09847709, 0.09847709]),
 array([0.08862938, 0.08862938, 0.08862938]),
 array(

In [127]:
p = -f_grad(b)
eta_1 = backtracking(eta_init, decay_rate, prop_constant, f, f_grad, b, p)
b_1 = b + eta_1*p
p_1 = -f_grad(b_1)
f_norm_1 = np.linalg.norm(-p)
eta_2 = backtracking(eta_init, decay_rate, prop_constant, f, f_grad, b_1, p_1)
b_2 = b_1 + eta_2*p_1
p_2 = -f_grad(b_2)
f_norm_2 = np.linalg.norm(-p_2)
eta_3 = backtracking(eta_init, decay_rate, prop_constant, f, f_grad, b_2, p_2)
b_3 = b_2 + eta_3*p_2
p_3 = -f_grad(b_3)
f_norm_3 = np.linalg.norm(-p_3)
eta_4 = backtracking(eta_init, decay_rate, prop_constant, f, f_grad, b_3, p_3)
b_4 = b_3 + eta_4*p_3
p_4 = -f_grad(b_4)
f_norm_4 = np.linalg.norm(-p_4)
eta_5 = backtracking(eta_init, decay_rate, prop_constant, f, f_grad, b_4, p_4)
b_5 = b_4 + eta_5*p_4
p_5 = -f_grad(b_5)
f_norm_5 = np.linalg.norm(-p_5)
eta_6 = backtracking(eta_init, decay_rate, prop_constant, f, f_grad, b_5, p_5)

NameError: name 'f_grad' is not defined

In [310]:
f_norm_5

0.0321426688539489

In [280]:
b_1

array([ 0.01024889,  0.00422772, -0.0023935 ])

In [283]:
f(b_1+eta_1*p)

0.859639283564846

In [247]:
np.linalg.norm(f_grad(beta_init))

18.212472424342383

In [251]:
p = -f_grad(b)
backtracking(eta_init, decay_rate, prop_constant, f, f_grad, b, p)

0.10737418240000006

In [127]:
eta_0 = 0.1
epsilon = 10**-2
lamb = 0.5
fastgradalgo(beta_init, eta_0, decay_rate, prop_constant, f, f_grad, epsilon, lamb)

eta:  1.6296287810675988e-11
eta:  1.7498005798264212e-12
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  

eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.59

eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.59

eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.59

eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.59

eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.5913e-320
eta:  5.59

KeyboardInterrupt: 

In [94]:
epsilon

1e-05

In [90]:
np.power(2,3)

8

In [81]:
def backtracking_test(eta_init, decay_rate, target_accuracy, f_grad, beta):
    
    eta = eta_init
    beta_t = beta
    theta_t = np.zeros(3)
    t = 0
    
    while not np.linalg.norm(f_grad(theta_t)) <= target_accuracy:
        beta_1 = theta_t - eta * f_grad(theta_t)
        print('beta_1: ', beta_1)
        theta_1 = beta_1 + np.dot((t/(t+3)),(beta_1 - beta_t))
        theta_t = theta_1
        beta_t = beta_1
        t += 1
        eta *= decay_rate
    
    return eta

In [119]:
backtracking(0.1, decay_rate, target_accuracy, f, f_grad, beta_init, -f_grad(beta_init))

0.051200000000000016

In [25]:
"""
beta_t = beta_init
betas = [beta_t]
theta_t = np.zeros(3)
eta = eta_init
t = 0

def f_grad(beta):

return computegrad(Xs, y_train, beta, 10)

while not np.linalg.norm(f_grad(theta_t)) <= target_accuracy:
beta_1 = theta_t - eta * f_grad(theta_t)
theta_1 = beta_1 + np.dot((t/(t+3)), (beta_1 - beta_t))
theta_t = theta_1
betas.append(beta_t)
beta_t = beta_1
t += 1
eta *= decay_rate

return betas

"""

'\nbeta_t = beta_init\nbetas = [beta_t]\ntheta_t = np.zeros(3)\neta = eta_init\nt = 0\n\ndef f_grad(beta):\n\nreturn computegrad(Xs, y_train, beta, 10)\n\nwhile not np.linalg.norm(f_grad(theta_t)) <= target_accuracy:\nbeta_1 = theta_t - eta * f_grad(theta_t)\ntheta_1 = beta_1 + np.dot((t/(t+3)), (beta_1 - beta_t))\ntheta_t = theta_1\nbetas.append(beta_t)\nbeta_t = beta_1\nt += 1\neta *= decay_rate\n\nreturn betas\n\n'

In [17]:
-f_grad(beta_init)

array([-0.00815028, -0.00815028, -0.00815028])

In [18]:
backtracking(eta_init, decay_rate, prop_constant, f, f_grad, beta_init, -f_grad(beta_init))

0.0032768000000000007

In [87]:
eta_init = 0.01
decay_rate = 0.8
target_accuracy = 0.001

def f_grad(beta):
    
    return computegrad(Xs, y_train, beta, 10)

beta_init = np.zeros(3)

#backtracking(eta_init, decay_rate, target_accuracy, f_grad, beta_init)

In [79]:
def get_eta(eta, decay_rate):
    
    eta *= decay_rate
    
    return eta

In [90]:
graddescent(beta_init, eta_init, target_accuracy, decay_rate)

[array([0., 0., 0.]),
 array([0., 0., 0.]),
 array([0.00016457, 0.00016457, 0.00016457]),
 array([0.00026892, 0.00026892, 0.00026892]),
 array([0.00036117, 0.00036117, 0.00036117]),
 array([0.00044006, 0.00044006, 0.00044006]),
 array([0.00050617, 0.00050617, 0.00050617]),
 array([0.00056091, 0.00056091, 0.00056091]),
 array([0.0006059, 0.0006059, 0.0006059]),
 array([0.00064274, 0.00064274, 0.00064274]),
 array([0.00067288, 0.00067288, 0.00067288]),
 array([0.00069754, 0.00069754, 0.00069754]),
 array([0.00071776, 0.00071776, 0.00071776]),
 array([0.00073439, 0.00073439, 0.00073439]),
 array([0.0007481, 0.0007481, 0.0007481])]

In [59]:
beta_1 = np.array([0.00082287, 0.00082287, 0.00082287])
beta_0 = np.array([0.,0.,0.])
np.dot((1/4),(beta_1 - beta_0))

array([0.00020572, 0.00020572, 0.00020572])

In [24]:
computegrad(Xs, y_train, beta_test, 10)

array([[-7.31367886e-01,  5.53271549e-04,  2.00829142e+00]])

In [38]:
def backtracking(eta_init, target_accuracy, f_grad, beta, theta):
    
    eta = eta_init
    t = 0
    betas = np.insert(beta, 0, 0)
    thetas = np.insert(theta, 0, 0)
    f_norm = np.linalg.norm(f_grad[t])

    
    while f_norm >= target_accuracy:
        
        eta = (thetas[t] - thetas[t+1] + (t/(t+3))*(betas[t+1]-betas[t]))/f_grad[t]
        t = t + 1
        f_norm = np.linalg.norm(f_grad[t])
        
    return eta

In [174]:
## <YOUR CODE HERE>
def backtracking(eta_init, decay_rate, prop_constant, f, f_grad, x, p):
    """Backtracking algorithm for stepsize selection.

	Parameters
	----------
	eta_init : float
		A positive number representing the starting, maximum stepsize.
    decay_rate : float
		A number in (0, 1) indicating how much to decrease eta while backtracking.
    prop_constant : float
		A number the proportionality between the amount of descent and directional derivative.
    f : function
		An objective function that maps a numpy array to a float.
    f_grad : function
		A function that computes the gradient of f at a point x. 
        Maps a 1D numpy array to a numpy array of the same size.
    x : numpy.ndarray
		A 1D array representing a point in d-dimensional space.
    p : numpy.ndarray
		A 1D array (unit vector) representing a direction in d-dimensional space.

	Returns
	-------
	eta : float
		The first eta value that satisfies the sufficient decrease condition.
	"""
    
    eta = eta_init
    
    def sufficient_decrease(eta):
        return f(x + eta * p) - f(x) <= prop_constant * eta * np.dot(f_grad(x), p)
    
    while not sufficient_decrease(eta):
        eta *= decay_rate
    
    return eta

In [175]:
def backtracking_line_search(x_init, eta_init, decay_rate, prop_constant, f, f_grad, max_iter=20):
    """Backtracking line search for optimization.

	Parameters
	----------
    x_init : numpy.ndarray
		Initial solution.
	eta_init : float
		A positive number representing the starting, maximum stepsize.
    decay_rate : float
		A number in (0, 1) indicating how much to decrease eta while backtracking.
    prop_constant : float
		A number the proportionality between the amount of descent and directional derivative.
    f : function
		An objective function that maps a numpy array to a float.
    f_grad : function
		A function that computes the gradient of f at a point x. 
        Maps a 1D numpy array to a numpy array of the same size.
    max_iter : int
		A positive integer containing the maximum number of iterations for the outer loop.

	Returns
	-------
	iterates : list
		A list of numpy arrays containing the iterates of the algorithm.
	"""
    x = x_init
    iterates = [x_init]

    for t in range(max_iter):
        
        # Pick search direction.
        p = -f_grad(x) ## <YOUR CODE HERE>
        
        # Pick stepsize.
        eta = backtracking(eta_init, decay_rate, prop_constant, f, f_grad, x, p) ## <YOUR CODE HERE>
        
        # Perform update.
        x = x + eta*p ## <YOUR CODE HERE>
        
        iterates.append(x)
        
    return iterates

In [176]:
def gradient_descent(x_init, eta, f_grad, max_iter=20):
    """Backtracking line search for optimization.

	Parameters
	----------
    x_init : numpy.ndarray
		Initial solution.
	eta : float
		Stepsize.
    f_grad : function
		A function that computes the gradient of f at a point x. 
        Maps a 1D numpy array to a numpy array of the same size.
    max_iter : int
		A positive integer containing the maximum number of iterations for the outer loop.

	Returns
	-------
	iterates : list
		A list of numpy arrays containing the iterates of the algorithm.
	"""
    x = x_init
    iterates = [x_init]

    for t in range(max_iter):
        x = x - eta * f_grad(x)        
        iterates.append(x)
        
    return iterates

def plot_f(f, iterates):
    nb = 100
    brange = np.linspace(-1.5, 1.5, nb)
    b1, b2 = np.meshgrid(brange, brange)
    z = np.array([f(x) for x in zip(b1.ravel(), b2.ravel())])
    
    levels=np.logspace(-2,5,30)
    
    plt.contour(brange, brange, z.reshape((nb, nb)), levels=levels);
    
    # Points.
    xy = np.array(iterates)
    plt.plot(xy[:,0], xy[:,1], 'k.');

    # Arrows.
    for j in range(1, len(iterates)):
        plt.annotate(
            "",
            xy=iterates[j],
            xytext=iterates[j - 1],
            arrowprops={"arrowstyle": "->", "color": "k", "lw": 1},
            va="center",
            ha="center",
        )

In [177]:
x_init = np.array([-1, 1])
eta = 0.01
max_iter = 100

iters_gd = gradient_descent(x_init, eta, f_grad, max_iter=max_iter)

plot_f(f, iters_gd)

TypeError: 'numpy.ndarray' object is not callable

In [111]:
frac = np.exp(-y_train*np.dot(beta_test, Xs.T))/(1+np.exp(-y_train*np.dot(beta_test, Xs.T)))
frac.shape

(1, 1000)

In [114]:
(frac * y_train).shape

(1, 1000)

In [173]:
np.sum(np.multiply(np.multiply(-y_train, frac), Xs.T))/len(Xs)+(2*5*beta_test)

array([[-0.73199176, -0.95979733,  0.11451564]])

In [127]:
Xs.shape

(1000, 3)

In [139]:
f_grad = computegrad(Xs, y_train, beta_test, 5)

In [148]:
np.linalg.norm(f_grad[0])

1.2124919304864115

In [158]:
thetas = np.insert(Xs, 0, 0)

In [159]:
betas = np.insert(beta_test, 0, 0)

In [166]:
betas[1]

-0.07320813310961584

In [167]:
betas[0]

0.0

In [168]:
thetas[0]

0.0

In [169]:
f_grad[0]

array([-0.73199176, -0.95979733,  0.11451564])

In [160]:
t=0
eta = (thetas[t] - thetas[t+1] + (t/(t+3))*(betas[t+1]-betas[t]))/f_grad[t]

In [165]:
(thetas[0]-thetas[1])

-1.444007029245418

In [39]:
backtracking(10, 0.0001, f_grad, beta_test, Xs)

NameError: name 'f_grad' is not defined

In [80]:
?np.linalg.norm

In [83]:
beta_test

array([[-0.06383013, -0.03734781,  0.12429325]])

In [90]:
betas = np.insert(beta_test, 0, 0)

In [92]:
betas[1]

-0.0638301265800845